# SQL Postgres n SQLITE

* KPIs in sql

somes refs:

    * customer retention: https://blog.statsbot.co/customer-retention-analysis-93af9daee46b
    * funnel analysis: https://blog.statsbot.co/sql-queries-for-funnel-analysis-35d5e456371d
    
        How many subscribe -> hoe many did audiction -> How many order an report
    
    * GAMES KPIs
    
        * https://applift.com/blog/user-retention
        * https://medium.com/n3twork/game-analytics-overview-ea8f53ffdff1
        * https://www.periscopedata.com/blog/how-to-calculate-cohort-retention-in-sql #A

* windows functions
* joins
* Filters 
* Transactions #A very important
* jaccard and trigrmas (at least postgres support that)

* How to insert csv file in sqlite3 in summarize data without worry about memory issues


In [4]:
import numpy as np
import pandas as pd

from scipy import stats

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

import IPython
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sqlalchemy import create_engine
import datetime as dt


## SQLITE

* Search how to run sql code in jupyter notebook cells
* terminal common command
* common SQL queries
    * joins
    * cross join (cartesian product)
* windowing functions (time series) GOOD explanation

### Code to process and add data to sqlite file


If the data does not fit in memmory and it is not big data, an alternative is to load the data in sqlite and use sqlite to summarize the data. In the case you need median and qunatiles you can take a large sample that fits in memmory and use python to compute that, so you will have a good estimation of these statistics.

refs: https://plot.ly/python/big-data-analytics-with-pandas-and-sqlite/

In [1]:
!head -n 1 data/logs.log

In [5]:
disk_engine = create_engine('sqlite:///logs.db') 

chunksize = 3
j = 0
start = dt.datetime.now()

file_name = 'data/logs.log'

columns =  [ 'date', 'time', 
            'site', 'computer_name', 'server_ip', 
            'request_type', 'uri_stem', 'uri_query', 'port', 
            'user_name', 'remote_ip', 'cs_version', 'browser', 'referer', 'host',
            'http_code', 'sub_status','win32_status',
            # https://support.symantec.com/en_US/article.TECH243050.html
            # cs-bytes: Number of bytes sent from client to server
            # sc-bytes: Number of bytes sent from server to client
            'sc_bytes','cs_bytes',
            'loading_time' ]

columns_to_drop = ['computer_name', 'uri_query', 'user_name', 'cs_version', 
                   'host', 
                   'sub_status','win32_status' ]

for df in pd.read_csv(file_name, sep=' ',index_col=False, header=None, chunksize=chunksize, iterator=True, encoding='utf-8'):
    
    print(df.shape)
    df.columns = columns
     
    # drop columns
    for c in columns_to_drop:
        df = df.drop(c, axis=1)
  
    # Do MORE processing in the columns

    # Insert data
    df.to_sql('log', disk_engine, if_exists='append')
    
    j+=1
    print('{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize))

    #assert j <= 3, "Break time"

(3, 21)
0 seconds: completed 3 rows
(3, 21)
0 seconds: completed 6 rows
(3, 21)
0 seconds: completed 9 rows
(2, 21)
0 seconds: completed 12 rows


In [8]:
!ls
!sqlite3 logs.db 'SELECT * FROM  log LIMIT 3;'

0|2018-10-01|07:00:16|W3SVC1574643156|10.0.1.50|GET|/castingworkbook/directors/director_home.asp|80|10.0.1.50|Mozilla/5.0+(Macintosh;+Intel+Mac+OS+X+10_12_6)+AppleWebKit/605.1.15+(KHTML,+like+Gecko)+Version/11.1.2+Safari/605.1.15|http://www.castingworkbook.com/castingworkbook/directors/dmenu2.asp|200|16171|1539|375
1|2018-10-01|07:00:16|W3SVC1574643156|10.0.1.50|GET|/castingworkbook/newFeatures.asp|80|10.0.1.50|Mozilla/5.0+(Macintosh;+Intel+Mac+OS+X+10_12_6)+AppleWebKit/605.1.15+(KHTML,+like+Gecko)+Version/11.1.2+Safari/605.1.15|http://www.castingworkbook.com/castingworkbook/directors/director_home.asp|200|189|1545|15
2|2018-10-01|07:00:18|W3SVC1574643156|10.0.1.50|GET|/castingworkbook/agents/shows.asp|80|10.0.1.50|Mozilla/5.0+(Macintosh;+Intel+Mac+OS+X+10_13_6)+AppleWebKit/605.1.15+(KHTML,+like+Gecko)+Version/12.0+Safari/605.1.15|http://www.castingworkbook.com/castingworkbook/agents/shows.asp?type=invitation|200|88306|1095|312


## Postgresq

* Search how to run sql code in jupyter notebook cells
* terminal common command
* common SQL queries
    * joins
    * cross join (cartesian product)
    * find the name of the special join that allows you in  a subquery use keyword defined outside
* KPIS (since is more likelly to use here and postgresql is more popular)
* windowing functions (time series) GOOD explanation
